# Настройка ноутбука

Любой исследовательский ноутбук должен начинаться с того, что мы устанавливаем необходимые пакеты для дальнейшей работы - не забывайте этого делать при создании ноутбуков для лабораторных! Делается это вот так:

In [ ]:
!pip install numpy==1.19.0
!pip install matplotlib==3.3.0
!pip install seaborn==0.11.0
!pip install pandas==1.0.0

In [ ]:
# Импорт необходимых модулей 
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = 'black'
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Линейная регрессия

Это наша первая работа, в которой мы коснемся реальных (ну почти) данных! Представльте себе, что к нам пришел интересующийся человек и хочет автоматически устанавливать цены на дома! И вот чудо, у него есть данные за последнее время, которые показывают пример того, как цены были установлены! Давайте постараемся помочь с этой нелегкой задачкой!

После того, как мы на практике познакомились с линейной регрессией - самое время применить ее на реальных данных. Для разработки и оценки будем использовать инструменты библиотеки `scikit-learn` (или сокращенно `sklearn`).

[Официальный сайт](https://scikit-learn.org/stable/) и [документация по API](https://scikit-learn.org/stable/modules/classes.html) (Application Program Interface - функции библиотеки) дают огромное количество полезной информации.

# Базовый анализ данных

В базовом анализе можно отметить следующие задачи:
- загрузка данных;
- ознакомление с данными, определение смысла переменных;
- поиск и устранение пропусков в данных;
- удаление неинформативных признаков;
- преобразование признаков к числовому виду (например, текст, аудио или картинка переводятся в числовое представление).

По сути главной целью базового анализа является минимальная подготовка данных для построения базовой модели (baseline). Минимальная подготовка - все данные привести к числовому виду, так как модели могут работать только с числовыми матрицами без пропусков.

## Загрузка данных

Для построения модели будет взят набор данных Boston House-Price, который представляет собой информацию о домах в Бостоне. Данный набор поставляется вместе с фреймворком `scikit-learn` и загружается следующим образом. Информацию о нем можно найти на официальном сайте https://scikit-learn.org/stable/datasets/index.html#boston-dataset.

In [ ]:
from sklearn.datasets import load_boston

boston_data = load_boston()
print(type(boston_data))

Переменная `boston_data` является типом (`Bunch`), похожим на тип `dict`, который хранит данные под ключами. В описании можно найти следующие важные ключи:
- `DESCR` - описание набора данных;
- `data` - данные (признаки) в формате матрицы 2D с размерностью [кол-во примеров, кол-во признаков];
- `feature_names` - названия признаков;
- `target` - значения целевой переменной для предсказания в формате вектора 1D по кол-ву примеров.

> Описание на сайте дает много дополнительной информации, но полученные ключи уже предоставляют необходимую информацию, так как `sklearn` хранит данные в собственном формате. После того, как будут получены данные, лучшей практикой является провести самостоятельный анализ для исключения возможных ошибок в описании.


Начнем с получения описания набора данных.

In [ ]:
print(boston_data['DESCR'])

Описание в загруженных данных во многом повторяет описание с сайта, но наиболее важными данными являются описания признаков в данных. 

> Одной из важных особенностей работы с данными является их "понимание". Без понимания признаков и целевой переменной анализ данных теряет возможность применения некоторых методик, которые позволяют обработать данные.

В данном датасете мы видим различные признаки, которые так или иначе описывают жилье. Всего заявлено 13 признаков (независимых переменных). Целевая переменная является медианным значение стоимости жилья.

Проверим размерности матрицы признаков и вектора предсказываемых значений. Так как они имеет тип `ndarray` (массив фреймворка `numpy`), то мы можем воспользоваться членом класса `.shape`, который является кортежем с элементами в виде размерности массива. Также, проверим тип данных через встроенную функцию Python - `type()`.

In [ ]:
feature_data = boston_data['data']
feature_names = boston_data['feature_names']
target_data = boston_data['target']

print(type(feature_data))
print(type(target_data))

print(feature_data.shape)
print(target_data.shape)

Отлично, размерности и тип соответствуют описанию. Теперь посмотрим, что содержит последний ключ.

In [ ]:
feature_names

Названия признаков соответсвуют описанию.

Для дальнейшей работы с данными рекомендуется перевести данные в формат фреймворка `pandas`, так как данные имеют матричный формат и работа в данном фреймворке позволит удобно организовать анализ.

In [ ]:
df = pd.DataFrame(feature_data, columns=boston_data['feature_names'])

df.head()

Для дальнейшего анализа включим значения целевых переменных в `DataFrame` под собственным именем.

In [ ]:
df['PRICE'] = target_data

df.head()

## Знакомство с данными

В ходе предобработки данных делается подготовка данных для дальнейшего использования модели. Наиболее частые причины подготовки табличных данных следующие:
- Пропуски данных в некоторых колонках (признаках)
- Очистка от выбросов в данных
- Исправление распределений, отличных от нормальных

Выведем базовую информацию по данным. С помощью методов `info()` и `describe()`.

In [ ]:
df.info()

In [ ]:
df.describe().T

## Определение наличия пропусков

Из результатов метода `info()` видно, что данные не имеют пропусков (количество non-null равно количеству записей в каждом признаке) - это важно, так как не требуется дополнять данные или исключать записи.

## Проверка и определение типов признаков

Помимо базовых функций также важно выделить разделение на типы признаков. Выделяют следующие основные типы признаков:
- Категориальные:
    - Номинальные - значения признаков ограничены группой возможных значений (красный/синий/зеленый);
    - Бинарные - те же номинальные, но всего две группы (Да/Нет, Правда/Ложь);
    - Последовательные - те же номинальные, но еще группы имеют порядок (плохой/хороший/отличный);
- Численные - вещественные или целочисленные данные.

Метод `df.info()` показал, что данные не имеют строк, которые сразу принимаются за категориальные признаки. На данный момент все признаки являются числовыми.

In [ ]:
df.nunique()

Признак `CHAS` имеет всего два значения - проверим, какие это значения:

In [ ]:
df['CHAS'].value_counts()

В данном случае можно рассматривать этот признак как категориальный, который не требует предобработки, так как он уже представлен числом.

Тем не менее обновим тип, чтобы в информации он был представлен верно:

In [ ]:
# Сначала приведем к int, чтобы вместо 0.0 и 1.0 были значения 0 и 1
# Затем приведем к типу категорий
df['CHAS'] = df['CHAS'].astype(int).astype(dtype='category')

df.info()

Остальные данные также можно привести к целочисленным, если они имеют только целочисленные значения, но делать это не обязательно.



Аналогично признак `RAD` имеет всего 9 значений, при этом по определению признака он является индексом, что может допускать в будущем и другие значения данного признака - так как нет явного ограничения, будем считать данный признак численным.

> Чаще всего выделение категориальных признаков делается исходя из априорных и предметных знаний, как, например, в анкете выпадающий список с вариантами. В данном датасете все признаки допускают диапазон вещественных/целочисленных значений. По этой причине в данной работе все признаки будут рассмотрены как численные.

## Поиск и удаление неинформативных признаков

Под неинформативными понимаются признаки, которые не несут полезной информации для предсказания. Это могут быть индексы записей в базе данных, полные ФИО пользователей или уникальный идентификатор каждого щеночка в базе. По сути это признаки, которые уникальны для каждой записи, поэтому на базовом этапе они удаляются. На этапе расширенного анализа они могут служить источником новых признаков.

В данном наборе данных судя по описанию мы не имеем уникальных для каждой записи признаков, так что удаление признаков не требуется.

## Результат

По результатам базового анализа и предоработки данные были приведены к числовому виду, так что можно их использовать для обучения базовой модели.

# Расширенный анализ данных

Расширенный (или описательный) анализ данных (Explanatory Data Analysis ~ EDA) является следующим этапом после построения базовой модели. Нацелен он на расширение знаний о данных, а также увеличение предсказательной способности (точности или требуемой метрики).

В качестве этапов можно выделить:
- более подходящяя обработка пропусков в данных;
- анализ распределений данных, исправление перекосов распределений;
- масштабирование значений признаков;
- корреляционный анализ;
- поиск новых зависимостей и генерация новых признаков;
- определение выбросов в данных;
- и другие подходы.

Сами по себе подходы не обязательно применяются все и разом. Одни подходы могут как улучшить результаты работы модели, так и ухудшить, поэтому данные подходы имеют экспериментальный характер.

В данной работе мы рассмотрим некоторые из подходов.

## Анализ распределений

Анализ распределений заключается в том, чтобы просмотреть распределения численных признаков, понять их характер и необходимость коррекции смещения.

Под нормальным (Гауссовым) распределением понимается распределение следующего вида:

In [ ]:
plt.figure(figsize=[10, 7])
sns.distplot(df['RM'], bins=20)
# Отключение меток на оси Y
plt.yticks([])
plt.show()

Смещение нормального распределения могут быть как левые (хвост находится справа), так и правое (хвост слева):

In [ ]:
_, ax = plt.subplots(nrows=1, ncols=2, figsize=[20, 7])

sns.distplot(df['DIS'], ax=ax[0], bins=20)
ax[0].title.set_text('Левое смещение')

sns.distplot(df['PTRATIO'], ax=ax[1], bins=20)
ax[1].title.set_text('Правое смещение')

plt.show()

Распределения могут иметь и бимодальный характер: 

In [ ]:
plt.figure(figsize=[10, 7])
sns.distplot(df['RAD'], bins=20)
# Отключение меток на оси Y
plt.yticks([])
plt.show()

Анализ распределений нужен во-первых, чтобы понять, как распределены данные по каждому признаку, и во-вторых, для понимания того, какие распределения требуют коррекции. Модель линейной регрессии чувствительна к тому, как данные распределены, поэтому коррекция смещения может помочь увеличить показатели работы модели. 

> Для этого `sklearn` имеет метод степенной коррекции `sklearn.preprocessing.PowerTransformer` и метод коррекции квантилями `sklearn.preprocessing.QuantileTransformer`.

## Корреляционный анализ

Корреляционный анализ важен для оценки того, как данные взаимосвязаны. В этом поможет функция отображения коэффициентов корреляции `DataFrame.corr()`. Корреляционный анализ позволит понять, какие переменные имеют связь с целевой переменной, что полезно для выбора признаков для обучения.


In [ ]:
# Для примера выберем следующие признаки
features = ['CRIM', 'NOX', 'RM', 'PRICE']

correlation_mtrx = df[features].corr()
correlation_mtrx

При этом отображение численных значений часто сложно для восприятия, поэтому проще отобразить данные в виде представления тепловой карты с помощью функции `heatmap()`.

In [ ]:
# Два дополнительных аргумента добавляют числа на график и указывают формат вывода данных
sns.heatmap(correlation_mtrx, annot=True, fmt='.2f')

Корреляционный анализ несет следующую полезную информацию - из выбранных признаков `RM` с целевой переменной `PRICE`, так как значение по модулю "достаточно высокое" (приблизительно больше 0.6).


### Визуальная оценка

После определения признаков, которые имеют высокую корреляцию, важно также посмотреть на распределение данных, так как показатель корреляции не сообщает о характере зависимости или о том, как данные распределены:

In [ ]:
sns.jointplot(x="RM", y="PRICE", data=df)

Как видно на графике, данные действительно имеют корреляцию: признак `RM` соответствует линейной зависимости.

> Сама по себе визуальная оценка взаимосвязи данных очень важна, так как по ней можно определить наличие зависимостей, при том, что показатель корреляции ориентируется лишь на линейную зависимость.

## Обработка выборосов данных

По ранее представленному графику важно отметить аномальное распределение точек по линии `PRICE = 50`, что говорит о возможных выбросах в данных. Такое же поведение видно и в графике распределения данных признака `PRICE`, что выражается в небольшой правой моде.

Для оценки количества возможных выбросов выберем данные, которые соответсвуют `PRICE = 50`.

In [ ]:
outliers_count = df[df['PRICE'] == 50].shape[0]

print(outliers_count)
print(outliers_count/df.shape[0]*100)

Оценка количества возможных выбросов (16 записей) показывает, что они составляют около трех процентов от всего количества данных, что не скажется на дальнейшей работе при их **исключении** из данных.

## Результат

По примерам методов анализа уже видно, что данные имеют стороны, которые можно постараться скорректировать, что может положительно сказаться на показателях работы модели.

# Разработка и оценка модели

## Разделение данных

Перед тем, как переходить к модели, требуется сделать разделение обраотанных данных на выборки для обучения и тестирования. Для этого воспользуется функцией `train_test_split()`.

> В ходе работы с данными важно сохранять свойство повторяемости. Для этого в функциях, использующих случайные генераторы, нужно использовать зафиксированное состояние генератора случайных чисел.

In [ ]:
from sklearn.model_selection import train_test_split

# 70% - для обучения, 30% - для тестов
TRAIN_RATIO = 0.7

# X - DataFrame только с признаками (для примера берем все признаки)
X = df[feature_names]
# y - истинные значения (разметка)
y = df['PRICE']

X_train, X_test, y_train, y_test = train_test_split(
    # Передаются наборы, которые будут разделены в соответсвии с параметрами
    X, y, 
    # Задается размер обучающей выборки
    train_size=TRAIN_RATIO, 
    # Фиксируется состояние генератора случайных чисел 
    #   для сохранения повторяемости
    random_state=RANDOM_STATE
)

# Отображение размеров выборок
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## Применение классов моделей

После того, как данные подготовлены, настало время создания модели, обучения и проверки.

Начнем с создания модели линейной регрессии и ее обучения с помощью метода `fit()`.

In [ ]:
from sklearn.linear_model import LinearRegression

# Создаем объект
reg = LinearRegression()
# Обучаем на выборке для обучения
reg.fit(X_train, y_train)

Для случая линейной регрессии обученная модель имеет веса для каждого признака. Объект класса `LinearRegression` позволяет получить эти веса и смещение путем обращения к атрибутам `.coef_` (веса признаков) и `.intercept_` (константное смещение, нулевой коэффициент). С аттрибутами класса модели линейной регрессии можно ознакомиться в справке здесь или на сайте https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html.

In [ ]:
# Для отображения весов создадим функцию
def show_linear_model_weights(model, feature_names=None):
    '''
    Отображение весов
    Аргументы:
        model - модель линейной регрессии
        feature_names - (опционально) если передано, отображение весов вместе с именами признаков
    '''
    print('\n----- Weights -----')
    weights = model.coef_
    bias = model.intercept_
    if feature_names is None:  
        print(f'Weights: {weights}')
    else:
        # Функция zip() делает из двух списков один список пар значений
        pairs = zip(feature_names, weights)
        for pair in pairs:
            print(f'{pair[0]} | {pair[1]}')

    print(f'Bias: {bias}')

show_linear_model_weights(reg, X_train.columns)

Каждый вес соответсвует коэффициенту, на который умножается значение признака, чтобы получить конечное значение предсказания. Так как данные не нормированны, мы не можем говорить о том, что данные веса отражают степень влияния признаков.

Для начала отобразим график предсказаний и посмотрим, как распределяются на линии пресказания. Для получения предсказания модели используется метод `predict()`.

In [ ]:
def draw_predictions_plot(y_pred, y_true):
    plt.scatter(y_pred, y_true)
    line_pnts = np.linspace(np.amin(y_pred)-0.1, np.amax(y_pred)+0.1, 1000)
    plt.plot(line_pnts, line_pnts, 'k--', lw=2)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.grid()

y_pred = reg.predict(X_test)
draw_predictions_plot(y_pred, y_test)

Прямая на графике означает точки, в которых предсказанные значения соответствуют действительным.
Как видно на графике, многие точки отклоняются от прямой, но наблюдается общая распределенность точек вокруг линии.


Помимо отображения точек предсказаний важно также оценить распределение ошибок, что может сказать о возможном смещении результатов.

In [ ]:
y_residuals = y_pred-y_test
sns.distplot(y_residuals, bins=50)
plt.grid()

Из графика распределения видно, что распределение имеет нормальный характер и центр распределения располагается возле значения 0. При этом имеются единичные отклонения, которые могут характеризоваться выбросами в данных, которые обсуждались ранее.


После представления точек предсказания важно также оценить численные характеристики. Для этого используются различные метрики. Мы воспользуемся наиболее распространенными: R2, Root Mean Squared Error (RMSE), Mean Absolute Error (MAE). 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# При повторении кода лучшей практикой является создание функции 
#   для исключения дублирований
# Дублирование кода -> ошибки
def evaluate_regressor(reg, X, y, mode):
    y_pred = reg.predict(X)
    r2_score_value = r2_score(y, y_pred)
    rmse_score = np.sqrt(mean_squared_error(y, y_pred))
    mae_score = mean_absolute_error(y, y_pred)

    print(f'\n----- Evaluation for {mode} -----')
    print(f'  R2: {r2_score_value}')
    print(f'  RMSE: {rmse_score}')
    print(f'  MAE: {mae_score}')

evaluate_regressor(reg, X_train, y_train, 'train')
evaluate_regressor(reg, X_test, y_test, 'test')

По результатам обучения и оценки мы получили **baseline** модель - это модель, которая требует минимум затрат сил и времени для получения первого результата. В данном случае были поданы все данные без очистки и предобработки. Последующие эксперименты нацелены на улучшение показателей (уменьшение MSE, MAE показателей или увеличение R2). 

## Применение классов трансформации

Одной из базовых практик предобработки численных переменных является **стандартизация**. Это процесс приведения распределения к нулевому среднему и единичному стандартному отклонению.

Для стандартизации данных применим класс `StandartScaler`, который в ходе обучения определяет среднее и стандартное отклонение каждого признака и далее применяет их в ходе использования модели.
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

При использовании стандартизации происходит определение необходимых параметров (среднее, стандартное отклонение) методом `fit()` и дальнейшая предобработка данных методом `transform()`.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
reg = LinearRegression()

# "Обучение" трансформера - определение параметров средних и отклонений
scaler.fit(X_train)

# Стандартизация данных
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

reg.fit(X_train_scaled, y_train)
show_linear_model_weights(reg, X_train.columns)

Важно обратить внимание на порядок весов модели. Использование стандартизации делает данные более центрированными и теперь веса больше отражают влияние на результат.

In [ ]:
# Отображаем средние значения и стандартные отклонения
# Количество значений соответствует количеству признаков
means = scaler.mean_
stds = scaler.var_

pairs = zip(X_test.columns, means, stds)
for pair in pairs:
    print(f'{pair[0]} | Mean: {pair[1]} | Std: {pair[2]}')

В результате работы с данными и применения различных методов обработки данных были получены базовые модели, описанные показателями метрик точности работы модели. На этом этапе мы кратко освоили базовые приемы, которые позволят провести исследование, предлагаемое в задании.

# Регуляризация

Помимо базового метода линейной регрессии существуют различные модификации. Одной из таких модификаций является метод Ридж (Ridge) регрессии. Основная идея Ридж регресси заключается в добавлении регуляризации по L2 норме. Функция потерь с Ридж регуляризацией выглядит следующим образом $$J=\|y-X^TW\|^2_2+\alpha*\|W\|^2_2$$

Как видно из функции потерь, минимизация происходит также с учетом величины значений весов, что и является регуляризацией. Регуляризация влияет на обучение путем удержания величины весов для исключения слишком сильного влияния признаков на работу модели, что на практике приводит к некорректным результатам.

# Задание

Требуется создать ноутбук, в котором будут проведены эксперименты с инструментами `sklearn`.

Задачи:
* Произведите базовую подготовку данных;
* Создайте baseline модель линейной регрессии;
---
* Произведите расширенный анализ данных:
    - Оцените корреляции признаков;
    - Оцените распределения признаков;
    - Оцените характер зависимости целевой переменной от каждого признака;
    - Напишите свои выводы по результатам анализа.

* Изучите работу инструментов нормализации данных:
    - Примените трансформер `sklearn.preprocessing.StandardScaler` к данным, отобразите распределения после обработки, обучите модель на обрабтанных данных и оцените показатели;
    - Примените трансформер `sklearn.preprocessing.MinMaxScaler` к данным, отобразите распределения после обработки, обучите модель на обрабтанных данных и оцените показатели;
    - Сравните результаты и сделайте выводы.

* Изучите работу инструментов выравнивания распределений:
    - Примените трансформер `sklearn.preprocessing.PowerTransformer` к данным, отобразите распределения после обработки, обучите модель на обработанных данных и оцените показатели;
    - Примените трансформер `sklearn.preprocessing.QuantileTransformer(output_distribution='normal')` к данным, отобразите распределения после обработки, обучите модель на обработанных данных и оцените показатели;
    - Сравните результаты и сделайте выводы.

* Исключите выбросы на уровне `PRICE == 50` из данных, обучите модель и оцените показатели;

* Оцените работу модели при исключении каждого признака [13 экспериментов] (результаты представить в виде таблицы `DataFrame` с указание исключенного признака и показателей метрик);

* Примените трансформер `sklearn.preprocessing.PolynomialFeatures` для генерации признака `LSTAT` в квадрате, обучите модель на данных с новым признаком, оцените работу модели;

---
* Создайте базовую модель Ридж регрессии https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html;

* Испытайте работу модели при использовании различных методов нормализации и выравнивания распределений;

* Оцените работу модели при различных $\alpha$ (изучить не менее 7 различных значений) и отобразите на графике (y-axis ~ R2, x-axis ~ log(alpha));

---
* Разработайте лучшую модель с необходимой предобработкой для решения задачи;
* Написать выводы по результатам.

> **Не забывайте разбивать ноутбук на секции, использовать функции, делать выполнение ноутбука последовательным.** 

# Вопросы

- Какую задачу выполняет `QuantileTransformer`?
- За что отвечает аргумент конструктора `output_distribution` класса `QuantileTransformer`?
- В чем отличие Ридж регрессии от линейной регрессии?
- Что такое регуляризация?
- Как сказывается станартизация данных на результатах обучения с использованием регуляризации?
- Как сказывается применение `QuantileTransformer` на результатах обучения с использованием регуляризации?
- Как влияет значение $\alpha$ на результаты обучения Ридж регрессии?